In [1]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.agents.sarsa import SARSAAgent
from rl.memory import EpisodeParameterMemory

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy
from rl.policy import EpsGreedyQPolicy
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [2]:
env = gym.make('fixed_treasure:fixed_treasure-v1')

print(env.observation_space)
print(env.action_space)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape

Box(2, 2)
Discrete(4)


In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(32, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(16, activation='relu'))
#model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

#sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=200, train_interval=50)
sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=0, train_interval=1)
sars.compile(optimizer='adam')

In [3]:
# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))
print(model.summary())

sars = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=0, train_interval=1)
sars.compile(optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
sars.fit(env, nb_steps=300000, visualize=False, verbose=1)

Training for 300000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 12:54 - reward: 0.0000e+00

/home/lkoziol/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 39s 4ms/step - reward: -0.9440
1012 episodes - episode_reward: -9.328 [-10.000, 10.000] - loss: 0.005 - mean_q: 0.283 - reward: -0.944

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 39s 4ms/step - reward: -0.9650
1009 episodes - episode_reward: -9.564 [-10.000, 10.000] - loss: 0.004 - mean_q: 0.250 - reward: -0.965

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 40s 4ms/step - reward: -0.9810
1005 episodes - episode_reward: -9.761 [-10.000, 10.000] - loss: 0.004 - mean_q: 0.250 - reward: -0.981

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 40s 4ms/step - reward: -0.9660
1008 episodes - episode_reward: -9.583 [-10.000, 10.000] - loss: 0.004 - mean_q: 0.250 - reward: -0.966

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: -0.9610
1011 episodes - episode_reward: -9.505 [-10.000, 1

In [5]:
sars.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: -10.000, steps: 10
Episode 2: reward: -10.000, steps: 10
Episode 3: reward: -10.000, steps: 10


TclError: invalid command name ".!frame.!canvas"